In [1]:
from IPython.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [35]:
import numpy as np
from desdeo_emo.EAs import RVEA, NSGAIII, IBEA
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_problem import DataProblem
from desdeo_tools.utilities import fast_non_dominated_sort, hypervolume_indicator

import matplotlib.pyplot as plt
import sklearn
from pyDOE import lhs
import pandas as pd

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, WhiteKernel, RationalQuadratic, DotProduct, ConstantKernel, Matern

from sklearn.gaussian_process import kernels
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.gaussian_process import GaussianProcessRegressor as GPR

## Assignment 2

Generate 100 samples (LHS sampling) with the welded beam design problem (K=2, n=4). 

Build two
types of surrogate models for each objective and ﬁnd the R-squared and RMSE for K fold cross
validations. 

Suggest which models to use (for each objective) for performing optimization based on
the accuracies. 

Finally generate 50 more LHS samples and test the models and show their RMSEs


- Use DESDEO, sklearn, GPy
- Optimize the hyperparameters for Kriging
- You can also try to optimize the hyperparameters for random forest

In [181]:
# the welded beam design problem
def f1(x):
    return 1.10471*x[:,0]**2*x[:,1] + 0.04811*x[:,2]*x[:,3]*(14 + x[:,1])


def f2(x):
    return 2.1952 / (x[:,2]**3*x[:,3])

The two objectives are the fabrication cost of the beam and the deflection of the end of the beam under the applied load P. The load P is fixed at 6,000 lbs, and the distance L is fixed at 14 in.

The design variables are:

x(1) = h, the thickness of the welds

x(2) = l, the length of the welds

x(3) = t, the height of the beam

x(4) = b, the width of the beam

Krigin and Random Forests were suggested.. 

In [195]:
def create_samples(samples):
    # create samples 
    x = lhs(4, samples)
    return x

## CREATE SAMPLES WITH LHS

In [245]:
x = create_samples(100)
func1 = f1(x)
func2 = f2(x)
f = np.vstack((func1, func2)).T

data = pd.DataFrame(np.hstack((x, f)), columns=["h", "l", "t", "b", "f1", "f2"])
f1_mean = func1.mean()
f2_mean = func2.mean()

print(f"Means of objective values with samples f1: {f1_mean} and f2: {f2_mean}")
data[:5]

Means of objective values with samples f1: 0.3731406834303446 and f2: 330053.74177119206


,h,l,t,b,f1,f2
0,0.696825,0.253083,0.335443,0.458088,0.241125,126.960957
1,0.494590,0.525668,0.219958,0.438255,0.209419,470.682244
2,0.013893,0.435320,0.900958,0.934106,0.584563,3.213398
3,0.444241,0.462126,0.152969,0.338337,0.136760,1812.640278
4,0.830599,0.795981,0.622840,0.545435,0.848468,16.657173


#### Optimizing the Gaussian Process Regressor hyperparameters

In [246]:
from sklearn.model_selection import train_test_split

x, x_test, f, y_test = train_test_split(x, f, test_size=0.20, random_state=7)

func1 = f[:,0] # train ys
func2 = f[:,1]

y1 = y_test[:,0] # test ys
y2 = y_test[:,1]

seed = 7

parameters = {
    "alpha": [1e-10, 1e-8, 1e-6, 1e-4,
                1e-2, 1e-0, 1e2, 1e4],
    "kernel": [
        None,
        kernels.RBF(length_scale_bounds=(1e-10, 1000000)),
        kernels.RationalQuadratic(length_scale_bounds=(1e-10, 1000000)),
        kernels.DotProduct(sigma_0_bounds=(1e-10, 1000000)),
        kernels.Matern(length_scale_bounds=(1e-10, 1000000)),
        kernels.ExpSineSquared(length_scale_bounds=(1e-10, 1000000))
    ]
}

# GridSearchCV does crossvalidation
gpr = GridSearchCV(GPR(normalize_y=True, n_restarts_optimizer=1,random_state=seed), parameters, cv=10)

gpr2 = GridSearchCV(GPR(normalize_y=True, n_restarts_optimizer=1,random_state=seed), parameters, cv=10)

# fit best parameters
gpr.fit(x, func1)
gpr = gpr.best_estimator_
print(gpr)

gpr2.fit(x, func2)
gpr2 = gpr2.best_estimator_
print(gpr2)

GaussianProcessRegressor(alpha=1e-08, n_restarts_optimizer=1, normalize_y=True,
                         random_state=7)
GaussianProcessRegressor(alpha=0.0001,
                         kernel=RationalQuadratic(alpha=1, length_scale=1),
                         n_restarts_optimizer=1, normalize_y=True,
                         random_state=7)


## F1 with GPR

In [247]:
m, std = gpr.predict(x_test, return_std=True)
print(mean_squared_error(m,y1, squared=False)) # dunno if this correct ?
print(r2_score(m, y1))

0.001547885830680775
0.999971946738293


## F1 with RF

In [248]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(random_state=7)
regr.fit(x, func1)
scores = cross_val_score(regr, x, func1, cv=10, scoring='r2')
print(scores)

rm = regr.predict(x_test)
print(mean_squared_error(rm,y1, squared=False))
print(r2_score(rm, y1))

[0.33336697 0.93872918 0.89232316 0.67054358 0.94119456 0.79931845
 0.84750489 0.7485616  0.9397964  0.62985609]
0.12152460972080045
0.5906373993857471


## F2 with GPR

In [249]:
m2, std2 = gpr2.predict(x_test, return_std=True)
print(mean_squared_error(m2, y2, squared=False))
print(r2_score(m2, y2))

6271405.057456365
-2006.2372673250013


## F2 with RF

In [250]:
regr2 = RandomForestRegressor(random_state=7)
regr2.fit(x, func2)

rscores2 = cross_val_score(regr2, x, func2, cv=10, scoring='r2')
print(rscores2)

rm2 = regr2.predict(x_test)
print(mean_squared_error(rm2,y2, squared=False))
print(r2_score(rm2, y2))

[-5.74140691e+04 -1.23527646e+02  6.41877574e-02  5.17031278e-01
  3.08241955e-01  7.84598943e-01 -6.55876011e+00  6.65939698e-01
 -2.02819595e+00 -1.19114788e+01]
6198801.45515619
-627.4329921390555


## Get test samples

In [251]:
# gen the test data

x_test = create_samples(50)
func_test1 = f1(x_test)
func_test2 = f2(x_test)
f1t_mean = func_test1.mean()
f2t_mean = func_test2.mean()
print(f"Means of test objective values with test samples f1: {f1t_mean} and f2: {f2t_mean}")
f_test = np.vstack((func_test1, func_test2)).T
test_data = pd.DataFrame(np.hstack((x_test, f_test)), columns=["h", "l", "t", "b", "f1", "f2"])
test_data[:5]

Means of test objective values with test samples f1: 0.3476898140281998 and f2: 40799.06137646955


,h,l,t,b,f1,f2
0,0.945195,0.049578,0.013712,0.620961,0.054686,1.371088e+06
1,0.325336,0.307963,0.896435,0.031593,0.055504,9.645516e+01
2,0.836416,0.786693,0.268481,0.535054,0.710185,2.120005e+02
3,0.880953,0.530463,0.957278,0.818375,1.002441,3.057788e+00
4,0.620299,0.203199,0.371361,0.864410,0.305722,4.958659e+01


## GPR F1 TEST

In [252]:
mean, std = gpr.predict(x_test, return_std=True)
print(mean_squared_error(mean,func_test1, squared=False)) # dunno if this correct ?
print(r2_score(mean, func_test1))

0.0030890540856077934
0.9998747587318525


## RF F1 TEST

In [253]:
rmean = regr.predict(x_test)
print(mean_squared_error(rmean,func_test1, squared=False)) # dunno if this correct ?
print(r2_score(rmean, func_test1))

0.1069419900405299
0.7593319507553078


## GPR F2 TEST

In [254]:
mean2, std2 = gpr2.predict(x_test,return_std=True)
print(mean_squared_error(mean2,func_test2, squared=False))
print(r2_score(mean2, func_test2))

230278.62035542377
-1.2078170020378547


## RF F2 TEST

In [255]:
rmean2 = regr2.predict(x_test)
print(mean_squared_error(rmean2,func_test2, squared=False))
print(r2_score(rmean2, func_test2))

152367.4892707121
0.5108065372310573


## Result 

|  |    Objective 1 GPR |  Objective 1 RF |  Objective 2 GPR | Objective 2 RF |
| --- | --- | --- | --- | --|
| Kfold CV R^2 | 0.999   | 0.591    | -2006.237   | -627.433  |  
| kfold CV RMSE | 0.002   | 0.122   | 6271405.057   | 6198801.455  |   
| Test R^2 | 0.999 |0.759 | -1.208 | 0.510  | 
| Test RMSE | 0.003 | 0.107 | 230278.620 | 152367.490   |    

### Some thoughts about the results

GPR seemed to handle objective 1 very well and easily within all crossvalidations in training and in the new generated test samples. While RF did good, GPR was superior in objective 1. RF had some worse results in training, more variance. Though to be noted, only hyperparameters of the GPR was optimized using GridSearchCV, RF could have reached better and most likely more stable results if used in the same way and then only use the best RF model in following tests too.

More interesting was objective 2, which had a lot of variance between its values. This can be seen on the worse results using both GPR and RF. RMSE was through the roof in many times, although both seemed to get around to the same magnitude of RMSE, many times $r^2$ values were very different. Like in the results, consistently RF showed better $r^2$ values. Results presented in the table are somewhat better runs, where the was less variance objective 2's values. This was just figured out by checking the mean.

In any case the results considering objective 2 varied a from a great range and only observation I got was that if the variance between the min and max of objective 2 values was smaller the methods worked bit better and the results had bit more sense. That's why run with "better results" was chosen in the end.